In [1]:
import pandas as pd
import numpy as np

import wget
import zipfile
import os

if not os.path.isfile("rawinventor.tsv"):
    wget.download("https://s3.amazonaws.com/data.patentsview.org/download/rawinventor.tsv.zip")
    with zipfile.ZipFile("rawinventor.tsv.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    os.remove("rawinventor.tsv.zip")

rawinventor = pd.read_csv("rawinventor.tsv", sep="\t")
rawinventor["mention_id"] = "US" + rawinventor.patent_id.astype(str) + "-" + rawinventor.sequence.astype(str)
disambiguation = rawinventor.set_index("mention_id")["inventor_id"]

def lambd(x):
    cluster = disambiguation[disambiguation == x.inventor_id].index.values
    to_add = np.setdiff1d(x["add"].split(", "), [""])
    cluster = np.append(cluster, to_add)
    to_remove = np.setdiff1d([string.strip() for string in x["remove"].split(",")], [""])
    if len(to_remove) > 0:
        assert all(mention in cluster for mention in to_remove), f"{to_remove[np.array([mention not in cluster for mention in to_remove])]}"
        cluster =  np.setdiff1d(cluster, to_remove)
    return cluster

## Emma validation

In [2]:
benchmark = pd.read_excel("2022-07-25-Emma-patent-samples-part-2.xlsx").fillna("")
true_clusters = benchmark.apply(lambd, axis=1)
reference = pd.concat({"inventor_id":benchmark.inventor_id, "mention_id":true_clusters}, axis=1).explode("mention_id").set_index("mention_id")["inventor_id"]

In [3]:
from pv_evaluation.benchmark import inspect_clusters_to_merge, style_cluster_inspection

style_cluster_inspection(inspect_clusters_to_merge(disambiguation, reference, join_with=rawinventor[["mention_id", "name_first", "name_last"]].set_index("mention_id")), by="reference")